In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [68]:
import seaborn as sns

In [69]:
sns.set_context('paper', font_scale=2.2)
sns.set_style('ticks')

In [70]:
import mako as m
import mako.template as mte
import mako.runtime as mrt
import StringIO as sio

In [84]:
dimension = 3

workgroup_x = 8
workgroup_y = 8
workgroup_z = 8

halo = 1 #ALWAYS 1 for this system.

num_alleles = 2

nx = 128
ny = 128
nz = 128

In [85]:
ctx_info = {}
ctx_info['num_type'] = 'double'


ctx_info['nx'] = nx
ctx_info['ny'] = ny
ctx_info['nz'] = nz

ctx_info['nx_bc'] = nx + 2*halo
ctx_info['ny_bc'] = ny + 2*halo
ctx_info['nz_bc'] = nz + 2*halo

ctx_info['halo'] = 1
ctx_info['buf_nx'] = workgroup_x + 2*halo
ctx_info['buf_ny'] = workgroup_y + 2*halo
ctx_info['buf_nz'] = workgroup_z + 2*halo

ctx_info['local_size'] = (workgroup_x, workgroup_y, workgroup_z)
ctx_info['domain_size'] = (nx, ny, nz)

ctx_info['dimension'] = 3
ctx_info['num_alleles'] = num_alleles

ctx_info['kernel_arguments'] = {}

In [86]:
import LB_colony_growth.LB_colony_growth as lb

In [87]:
lb.DLA_Colony(ctx_info)

global size: (128, 128, 128)
local size: (8, 8, 8)
The platforms detected are:
---------------------------
Intel(R) OpenCL Intel(R) Corporation version: OpenCL 1.2 LINUX
The devices detected on platform Intel(R) OpenCL are:
---------------------------
       Intel(R) Core(TM) i7-3537U CPU @ 2.00GHz [Type: CPU ]
Maximum clock Frequency: 2000 MHz
Maximum allocable memory size: 2062 MB
Maximum work group size 8192
Maximum work item dimensions 3
Maximum work item size [8192L, 8192L, 8192L]
---------------------------
This context is associated with  1 devices


UnboundLocalError: local variable 'bc_map' referenced before assignment

In [59]:
template = mte.Template(
    filename='./LB_colony_growth/colony_growth.mako',
    strict_undefined=True
)

In [60]:
buf = sio.StringIO()

context = mrt.Context(buf, **ctx_info)
template.render_context(context)

print buf.getvalue()

#ifdef cl_khr_fp64
    #pragma OPENCL EXTENSION cl_khr_fp64 : enable
#elif defined(cl_amd_fp64)
    #pragma OPENCL EXTENSION cl_amd_fp64 : enable
#else
    #error "Double precision floating point not supported by OpenCL implementation."
#endif

// Define domain size
#define nx 128
#define ny 128
#define nz 128
// Define boundary map domain size
#define nx_bc 130
#define ny_bc 130
#define nz_bc 130

#define SMALL 1e-6

#define NUM_NEAREST_NEIGHBORS 6
__constant int cx_nearest[6] = {1, -1, 0, 0, 0, 0};
__constant int cy_nearest[6] = {0,  0, 1,-1, 0, 0};
__constant int cz_nearest[6] = {0,  0, 0, 0, 1,-1};

//The code is always ok, AS LONG as the halo is one! Regardless of the stencil.
// If any more, everything breaks.
#define halo 1

#define FLUID_NODE 0
#define WALL_NODE 1
#define NOT_IN_DOMAIN 2
//Alleles get negative numbers as identifiers
#define ALLELE_1 -1
#define ALLELE_2 -2

inline int get_spatial_index_2(
    const int x, const int y,
    const int x_size, const int y_size)
{
  

In [29]:
ctx_info['kernel_arguments']

{'collide_and_propagate': [['bc_map',
   '__global __read_only int *bc_map_global'],
  ['num_jumpers', 'const int num_jumpers'],
  ['f', '__global double *f_global'],
  ['feq', '__global __read_only double *feq_global'],
  ['omega', 'const double omega'],
  ['c_vec', '__constant int *c_vec'],
  ['c_mag', '__constant double *c_mag'],
  ['w', '__constant double *w'],
  ['num_jumpers', 'const int num_jumpers'],
  ['rho', '__global double *rho_global'],
  ['buf_nx', 'const int buf_nx'],
  ['buf_ny', 'const int buf_ny'],
  ['buf_nz', 'const int buf_nz'],
  ['local_mem', '__local double *rho_local'],
  ['local_mem', '__local double *bc_map_local'],
  ['k', 'const double k'],
  ['D', 'const double D']],
 'reproduce': [],
 'update_after_streaming': []}